In [2]:
import json
import sys
import math
from tqdm import tqdm
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import copy
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn import svm
import pickle

## LDA_100

In [20]:
train_path = 'train_data/all_train_100.csv'
train_data = pd.read_csv(train_path)
train_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,2.5,1.0,0.0,0.0,1.0,1.0,0.0,...,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204
1,108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,3.5,1.0,0.0,0.0,0.0,0.0,1.0,...,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189
2,300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,4.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526
3,318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,3.5,1.0,0.0,1.0,1.0,0.0,1.0,...,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.039897,0.000385
4,331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,3.5,0.0,0.0,0.0,0.0,0.0,1.0,...,0.001111,0.001111,0.223333,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111


In [21]:
test_path = 'train_data/all_test_100.csv'
test_data = pd.read_csv(test_path)
test_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,3.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.000012,0.000027,0.000005,9.320489e-07,0.000043,9.320489e-07,0.000002,0.000013,0.000032,0.000012
1,499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,4.5,1.0,1.0,1.0,1.0,0.0,0.0,...,0.098289,0.000005,0.000791,4.591333e-06,0.000017,4.591333e-06,0.001446,0.000005,0.000943,0.000019
2,533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,4.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.027581,0.000005,0.000386,4.700108e-06,0.000059,4.700108e-06,0.000846,0.000042,0.001169,0.000064
3,190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.001919,0.000024,0.000013,5.558423e-06,0.000086,5.577801e-06,0.000010,0.000086,0.000042,0.001630
4,394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.001160,0.000020,0.000016,4.482868e-06,0.000015,4.482868e-06,0.000009,0.000050,0.000113,0.001357


In [22]:
train_x = train_data.dropna()
train_y = np.array(train_x['Ratings']).astype(float).ravel()
del train_x['Ratings']
del train_x['business_id']
del train_x['user_id']
del train_x['Unnamed: 0']
test_x = test_data.dropna()
test_y = np.array(test_x['Ratings']).astype(float).ravel()
del test_x['Ratings']
del test_x['business_id']
del test_x['user_id']
del test_x['Unnamed: 0']

In [23]:
train_x = train_x.iloc[:,9:]
train_x

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,...,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204
1,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.187808,0.000189,0.000189,0.000189,...,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189
2,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.081599,0.000526,0.000526,0.000526,...,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526
3,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.177176,0.000385,0.000385,0.184194,...,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.039897,0.000385
4,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,...,0.001111,0.001111,0.223333,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248363,0.000105,0.036345,0.000105,0.000105,0.000105,0.000105,0.285796,0.028640,0.000105,0.000105,...,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.021158,0.010656,0.000105
248364,0.000164,0.000164,0.000164,0.000164,0.000164,0.000164,0.112673,0.095196,0.000164,0.000164,...,0.000164,0.000164,0.000164,0.000164,0.000164,0.000164,0.000164,0.032951,0.000164,0.000164
248365,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.362314,0.000370,0.000370,0.000370,...,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370
248366,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,...,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385


In [24]:
test_x = test_x.iloc[:,9:]
test_x

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,0.000028,0.000011,0.000005,0.000003,9.542065e-07,0.000006,0.126199,0.000073,0.000001,0.000199,...,0.000012,0.000027,0.000005,9.320489e-07,0.000043,9.320489e-07,0.000002,0.000013,0.000032,0.000012
1,0.000013,0.000024,0.000016,0.000007,4.591333e-06,0.000023,0.067429,0.000109,0.000006,0.009839,...,0.098289,0.000005,0.000791,4.591333e-06,0.000017,4.591333e-06,0.001446,0.000005,0.000943,0.000019
2,0.000005,0.000005,0.000016,0.000005,4.700108e-06,0.000005,0.050177,0.000366,0.000005,0.022236,...,0.027581,0.000005,0.000386,4.700108e-06,0.000059,4.700108e-06,0.000846,0.000042,0.001169,0.000064
3,0.000043,0.000019,0.000018,0.000014,5.480910e-06,0.000020,0.001043,0.000405,0.000006,0.000419,...,0.001919,0.000024,0.000013,5.558423e-06,0.000086,5.577801e-06,0.000010,0.000086,0.000042,0.001630
4,0.000011,0.000023,0.000027,0.000006,4.482868e-06,0.000004,0.001078,0.000306,0.000004,0.000167,...,0.001160,0.000020,0.000016,4.482868e-06,0.000015,4.482868e-06,0.000009,0.000050,0.000113,0.001357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138142,0.000167,0.001275,0.000003,0.000003,2.980672e-06,0.000003,0.073495,0.001063,0.000003,0.000165,...,0.002136,0.000068,0.000003,2.980672e-06,0.000547,2.980672e-06,0.000003,0.000357,0.000003,0.000112
138143,0.000006,0.000245,0.000003,0.000001,1.300730e-06,0.000006,0.343546,0.006841,0.000001,0.000547,...,0.000019,0.000194,0.000001,1.300730e-06,0.000003,1.300730e-06,0.000001,0.000475,0.000341,0.000087
138144,0.000005,0.000400,0.000002,0.000002,1.847042e-06,0.000020,0.174643,0.006262,0.000002,0.000264,...,0.000027,0.000141,0.000002,1.847042e-06,0.000005,1.847042e-06,0.000002,0.001541,0.001171,0.000219
138145,0.000011,0.000425,0.000006,0.000004,1.627046e-06,0.000004,0.277236,0.029862,0.000002,0.001433,...,0.000078,0.000050,0.000003,1.627046e-06,0.000030,1.627046e-06,0.000002,0.002493,0.000537,0.000104


In [28]:
def get_round_number(x):
    if x > 5:
        return 5
    if x < 0.5:
        return 0.5
    a = int(x)
    b = x - a
    if b < 0.25:
        return float(a)
    elif b > 0.75:
        return float(a+1)
    else:
        return a + 0.5

slr = LinearRegression()
model = slr.fit(train_x, train_y)
y = slr.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('Linear Regression: ', np.sqrt(mean_squared_error(y_pred, test_y)))

Linear Regression:  0.6491859337653892


## LDA_50

In [3]:
train_path = 'train_data/all_train_50.csv'
train_data = pd.read_csv(train_path)
train_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,topic_41,topic_42,topic_43,topic_44,topic_45,topic_46,topic_47,topic_48,topic_49,topic_50
0,108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,2.5,1.0,0.0,0.0,1.0,1.0,0.0,...,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.020571,0.083094
1,108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,3.5,1.0,0.0,0.0,0.0,0.0,1.0,...,0.000377,0.000377,0.000377,0.000377,0.000377,0.020893,0.000377,0.000377,0.000377,0.000377
2,300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,4.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053
3,318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,3.5,1.0,0.0,1.0,1.0,0.0,1.0,...,0.000769,0.000769,0.039231,0.000769,0.000769,0.000769,0.000769,0.000769,0.000769,0.132505
4,331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,3.5,0.0,0.0,0.0,0.0,0.0,1.0,...,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222


In [4]:
test_path = 'train_data/all_test_50.csv'
test_data = pd.read_csv(test_path)
test_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,topic_41,topic_42,topic_43,topic_44,topic_45,topic_46,topic_47,topic_48,topic_49,topic_50
0,108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,3.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.000390,0.000017,0.000015,0.000003,0.000037,0.000368,0.000116,0.000017,0.000730,0.004353
1,499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,4.5,1.0,1.0,1.0,1.0,0.0,0.0,...,0.000088,0.000012,0.001105,0.000007,0.000039,0.000023,0.000298,0.000018,0.000031,0.003686
2,533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,4.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000289,0.000038,0.001584,0.000008,0.000052,0.000015,0.000358,0.000034,0.000008,0.004278
3,190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.000185,0.006823,0.000040,0.000013,0.000085,0.001181,0.001187,0.000427,0.000086,0.000065
4,394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.000247,0.001771,0.000419,0.000017,0.000063,0.001751,0.001892,0.000312,0.000091,0.000070


In [5]:
train_x = train_data.dropna()
train_y = np.array(train_x['Ratings']).astype(float).ravel()
del train_x['Ratings']
del train_x['business_id']
del train_x['user_id']
del train_x['Unnamed: 0']
test_x = test_data.dropna()
test_y = np.array(test_x['Ratings']).astype(float).ravel()
del test_x['Ratings']
del test_x['business_id']
del test_x['user_id']
del test_x['Unnamed: 0']

In [6]:
train_x = train_x.iloc[:,9:]
train_x

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_41,topic_42,topic_43,topic_44,topic_45,topic_46,topic_47,topic_48,topic_49,topic_50
0,0.000408,0.000408,0.000408,0.000408,0.000408,0.024690,0.000408,0.000408,0.000408,0.000408,...,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.020571,0.083094
1,0.000377,0.000377,0.000377,0.374823,0.000377,0.000377,0.000377,0.000377,0.000377,0.000377,...,0.000377,0.000377,0.000377,0.000377,0.000377,0.020893,0.000377,0.000377,0.000377,0.000377
2,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,...,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053
3,0.000769,0.000769,0.000769,0.000769,0.000769,0.000769,0.000769,0.000769,0.000769,0.000769,...,0.000769,0.000769,0.039231,0.000769,0.000769,0.000769,0.000769,0.000769,0.000769,0.132505
4,0.002222,0.002222,0.120918,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,...,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248363,0.000211,0.000211,0.010998,0.143911,0.000211,0.000211,0.000211,0.000211,0.000211,0.021263,...,0.000211,0.000211,0.000211,0.000211,0.000211,0.037729,0.000211,0.000211,0.000211,0.031303
248364,0.000328,0.000328,0.000328,0.000328,0.000328,0.027574,0.000328,0.000328,0.000328,0.000328,...,0.000328,0.016721,0.000328,0.000328,0.025800,0.000328,0.000328,0.000328,0.000328,0.082565
248365,0.000741,0.000741,0.037778,0.000741,0.000741,0.000741,0.000741,0.000741,0.000741,0.000741,...,0.000741,0.000741,0.000741,0.000741,0.000741,0.000741,0.000741,0.000741,0.000741,0.000741
248366,0.000769,0.000769,0.286308,0.000769,0.000769,0.056308,0.000769,0.000769,0.000769,0.077802,...,0.000769,0.000769,0.000769,0.000769,0.038886,0.000769,0.000769,0.000769,0.000769,0.000769


In [7]:
test_x = test_x.iloc[:,9:]
test_x

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_41,topic_42,topic_43,topic_44,topic_45,topic_46,topic_47,topic_48,topic_49,topic_50
0,0.000018,0.000036,0.000020,0.337294,0.000003,0.017139,0.000003,0.000037,0.000006,0.000043,...,0.000390,0.000017,0.000015,0.000003,0.000037,0.000368,0.000116,0.000017,0.000730,0.004353
1,0.000010,0.000022,0.000648,0.000385,0.000007,0.000067,0.000008,0.000066,0.000014,0.000030,...,0.000088,0.000012,0.001105,0.000007,0.000039,0.000023,0.000298,0.000018,0.000031,0.003686
2,0.000008,0.000122,0.001231,0.002278,0.000008,0.000008,0.000008,0.000008,0.000008,0.000014,...,0.000289,0.000038,0.001584,0.000008,0.000052,0.000015,0.000358,0.000034,0.000008,0.004278
3,0.000049,0.000043,0.000040,0.002351,0.000015,0.000082,0.000014,0.000028,0.000032,0.000057,...,0.000185,0.006823,0.000040,0.000013,0.000085,0.001181,0.001187,0.000427,0.000086,0.000065
4,0.000028,0.000015,0.000034,0.001061,0.000012,0.000148,0.000013,0.000027,0.000051,0.000054,...,0.000247,0.001771,0.000419,0.000017,0.000063,0.001751,0.001892,0.000312,0.000091,0.000070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138142,0.001407,0.000027,0.001830,0.008632,0.000044,0.000089,0.000010,0.000422,0.000010,0.000026,...,0.000796,0.000026,0.000035,0.000010,0.000186,0.004443,0.003277,0.000095,0.000667,0.001445
138143,0.000128,0.000040,0.000964,0.076997,0.000004,0.000868,0.000004,0.000009,0.000006,0.001528,...,0.000036,0.000147,0.000005,0.000004,0.001153,0.000619,0.000498,0.000042,0.000033,0.002158
138144,0.000051,0.000018,0.000565,0.077812,0.000004,0.001131,0.000004,0.000006,0.000004,0.001651,...,0.000119,0.000076,0.000043,0.000006,0.000580,0.000680,0.000250,0.000228,0.000028,0.002153
138145,0.000185,0.000018,0.001591,0.062286,0.000005,0.001151,0.000007,0.000015,0.000010,0.001146,...,0.000191,0.000143,0.000049,0.000006,0.001229,0.001072,0.000965,0.000116,0.000063,0.006592


In [8]:
def get_round_number(x):
    if x > 5:
        return 5
    if x < 0.5:
        return 0.5
    a = int(x)
    b = x - a
    if b < 0.25:
        return float(a)
    elif b > 0.75:
        return float(a+1)
    else:
        return a + 0.5

slr = LinearRegression()
model = slr.fit(train_x, train_y)
y = slr.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('Linear Regression: ', np.sqrt(mean_squared_error(y_pred, test_y)))

Linear Regression:  0.6505017382414816


## TF-IDF

In [9]:
train_path = 'train_data/tfidf_train.csv'
train_data = pd.read_csv(train_path)
train_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,went,wife,wine,wish,wo,wonder,work,worth,wrong,year
0,108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,2.5,1.0,0.0,0.0,1.0,1.0,0.0,...,0.148474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,3.5,1.0,0.0,0.0,0.0,0.0,1.0,...,0.142877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,4.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,3.5,1.0,0.0,1.0,1.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.294119
4,331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,3.5,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [10]:
test_path = 'train_data/tfidf_test.csv'
test_data = pd.read_csv(test_path)
test_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,went,wife,wine,wish,wo,wonder,work,worth,wrong,year
0,108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,3.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.145676,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
1,499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,4.5,1.0,1.0,1.0,1.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.073530
2,533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,4.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.073530
3,190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.000000,0.0,0.0,0.077334,0.0,0.0,0.059389,0.259429,0.0,0.066885
4,394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.077334,0.0,0.0,0.059389,0.259429,0.0,0.066885


In [11]:
train_x = train_data.dropna()
train_y = np.array(train_x['Ratings']).astype(float).ravel()
del train_x['Ratings']
del train_x['business_id']
del train_x['user_id']
del train_x['Unnamed: 0']
test_x = test_data.dropna()
test_y = np.array(test_x['Ratings']).astype(float).ravel()
del test_x['Ratings']
del test_x['business_id']
del test_x['user_id']
del test_x['Unnamed: 0']

In [12]:
train_x = train_x.iloc[:,9:]
train_x

,10,15,20,30,abl,absolut,actual,alway,amaz,anoth,...,went,wife,wine,wish,wo,wonder,work,worth,wrong,year
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.148474,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.142877,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285026,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.306331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.249899,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.294119
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248363,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.105121,0.0,0.300430,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
248364,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.125060,0.0,0.357413,0.0,0.0,0.160329,0.0,0.0,0.0,0.000000
248365,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
248366,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.225635,0.0,0.0,0.0,0.000000


In [13]:
test_x = test_x.iloc[:,9:]
test_x

,10,15,20,30,abl,absolut,actual,alway,amaz,anoth,...,went,wife,wine,wish,wo,wonder,work,worth,wrong,year
0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,...,0.145676,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,0.076583,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.133731,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.073530
2,0.076583,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.133731,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.073530
3,0.000000,0.0,0.0,0.0,0.0,0.100316,0.0,0.07687,0.000000,0.0,...,0.000000,0.0,0.000000,0.077334,0.000000,0.000000,0.059389,0.259429,0.0,0.066885
4,0.000000,0.0,0.0,0.0,0.0,0.100316,0.0,0.07687,0.000000,0.0,...,0.000000,0.0,0.000000,0.077334,0.000000,0.000000,0.059389,0.259429,0.0,0.066885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138444,0.000000,0.0,0.0,0.0,0.0,0.021087,0.0,0.03069,0.138948,0.0,...,0.039695,0.0,0.000000,0.000000,0.000000,0.000000,0.015309,0.000000,0.0,0.019372
138445,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,...,0.038363,0.0,0.109641,0.000000,0.024983,0.064327,0.000000,0.000000,0.0,0.052574
138446,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,...,0.038363,0.0,0.109641,0.000000,0.024983,0.064327,0.000000,0.000000,0.0,0.052574
138447,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,...,0.038363,0.0,0.109641,0.000000,0.024983,0.064327,0.000000,0.000000,0.0,0.052574


In [14]:
def get_round_number(x):
    if x > 5:
        return 5
    if x < 0.5:
        return 0.5
    a = int(x)
    b = x - a
    if b < 0.25:
        return float(a)
    elif b > 0.75:
        return float(a+1)
    else:
        return a + 0.5

slr = LinearRegression()
model = slr.fit(train_x, train_y)
y = slr.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('Linear Regression: ', np.sqrt(mean_squared_error(y_pred, test_y)))

Linear Regression:  0.7106322750869876
